In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder

import warnings
warnings.filterwarnings('ignore')

# Data Load

In [ ]:
# Data load
train = pd.read_csv('../input/home-credit-default-risk/application_train.csv')
test = pd.read_csv('../input/home-credit-default-risk/application_test.csv')

In [ ]:
# See head data of application_train
train.head()

In [ ]:
# See describe of application_train
train.describe()

In [ ]:
# See head data of application_test
test.head()

### 기본적으로 train에는 TARGET이 있고 test에는 TARGET column이 없으므로 column 수에서 1 차이가 난다.

In [ ]:
train.columns.values

## Data exploration

In [ ]:
# TARGET data의 분포 barplot으로 시각화

target_df = train["TARGET"].value_counts()
target_vc_df = pd.DataFrame({'class': target_df.index, 'values': target_df.values})

plt.figure(figsize = (6,6))
plt.title('Application loans repayed - train dataset')
sns.barplot(x = 'class', y="values", data=target_vc_df)
locs, labels = plt.xticks(ticks=[0,1], labels=["repay", "not repay"])
plt.show()

In [ ]:
# NAME_CONTRACT_TYPE과 TARGET을 countplot으로 시각화

sns.countplot(x='NAME_CONTRACT_TYPE', data=train, hue='TARGET')

In [ ]:
# FLAG_OWN_CAR과 TARGET을 countplot으로 시각화

sns.countplot(x='FLAG_OWN_CAR', data=train, hue='TARGET')

In [ ]:
# 그래프의 정보를 value과 percentage로 모두 보여주는 함수

def plot_stats(feature,label_rotation=False,horizontal_layout=True):
    temp = train[feature].value_counts()
    df1 = pd.DataFrame({feature: temp.index,'Number of contracts': temp.values})

    # Calculate the percentage of target=1 per category value
    cat_perc = train[[feature, 'TARGET']].groupby([feature],as_index=False).mean()
    cat_perc.sort_values(by='TARGET', ascending=False, inplace=True)
    
    if(horizontal_layout):
        fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(12,6))
    else:
        fig, (ax1, ax2) = plt.subplots(nrows=2, figsize=(12,14))
    sns.set_color_codes("pastel")
    s = sns.barplot(ax=ax1, x = feature, y="Number of contracts",data=df1)
    if(label_rotation):
        s.set_xticklabels(s.get_xticklabels(),rotation=90)
    
    s = sns.barplot(ax=ax2, x = feature, y='TARGET', order=cat_perc[feature], data=cat_perc)
    if(label_rotation):
        s.set_xticklabels(s.get_xticklabels(),rotation=90)
    plt.ylabel('Percent of target with value 1 [%]', fontsize=10)
    plt.tick_params(axis='both', which='major', labelsize=10)

    plt.show();

In [ ]:
# NAME_CONTRACT_TYPE에 대해 비율과 value count

plot_stats('NAME_CONTRACT_TYPE')

In [ ]:
# NAME_CONTRACT_TYPE에 대해 비율과 value count

plot_stats('CNT_CHILDREN')

# TARGET 값이 1인 data들 중에는 childeren count의 비율이 9, 11이 가장 크다.
# 부양 가족이 많기 때문에 대출 상환에 어려움을 겪을 수도 있다.

## EDA

In [ ]:
# TARGET data의 분포를 다른 그래프로도 추가 시가화

figure ,ax = plt.subplots(1, 2, figsize=(12,5))

train['TARGET'].value_counts().plot.pie(explode=[0,0.1],autopct='%1.1f%%', ax=ax[0], shadow=True)
ax[0].set_title('TARGET')
ax[0].set_ylabel('')

sns.countplot(x='TARGET', data=train, ax=ax[1])
ax[1].set_title('TARGET')

plt.show()

In [ ]:
# EDA_1) dataframe에서 missing value가 얼마나 있는지 전체에 비해 몇 퍼센트의 비율인지 확인하는 함수
def missing_data(data):
    total = data.isnull().sum().sort_values(ascending=False)
    percent = (data.isnull().sum() / data.isnull().count() * 100).sort_values(ascending=False)
    return pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])

In [ ]:
print(missing_data(train))

In [ ]:
# EDA_2) train data에서 missing value가 있는 것들만 print
missing_data(train)[missing_data(train)['Total']!=0]

In [ ]:
# test data에 대해서도 적용
print(missing_data(test))

In [ ]:
# EDA_3)  test data에서 missing value가 있는 것들만 print
missing_data(test)[missing_data(test)['Total']!=0]

In [ ]:
# 전체 data들의 type 확인
train.dtypes.value_counts()

In [ ]:
# EDA_4) 범주형 변수(object)에서 unique 값의 개수 확인
train.select_dtypes('object').nunique()

## FE

In [ ]:
# FE_1) 범주형 변수의 처리를 위해 Label Encoding 진행

le=LabelEncoder()
le_count=0
encoded_col = []

for col in train:
    if train[col].dtype=='object':
        # data type이 object이고 값의 종류가 두개 이하일경우,
        if len(list(train[col].unique())) <=2:
            le.fit(train[col])
            
            train[col]=le.transform(train[col])
            test[col]=le.transform(test[col])
            
            encoded_col.append(col)
            le_count+=1
print('Label Encoding column 수 : %d' % le_count)
print('Encoded columns :', encoded_col)

In [ ]:
# FE_2) Label Encoding을 진행하지 않은 나머지 범주형 변수에 대해 one-hot encoding 진행
train = pd.get_dummies(train)
test = pd.get_dummies(test)

print('Training shape: ', train.shape)
print('Testing shape: ', test.shape)

In [ ]:
# FE_3) train과 test set을 align 시키는 작업

# train의 TARGET 변수는 test에 없어서 미리 추출
train_target = train['TARGET']

# train과 test에 모두 있는 column들을 기준으로 align
train, test = train.align(test, join = 'inner', axis = 1)

# 사전에 추출해 둔 TARGET 변수 추가
train['TARGET'] = train_target

print('Training shape: ', train.shape)
print('Testing shape: ', test.shape)

## Check Anomaly

In [ ]:
# EDA_5) DAYS_BIRTH의 anomaly check
(train['DAYS_BIRTH'] / -365).describe()

# No anomaly

In [ ]:
# EDA_6) check DAYS_EMPLOYED
train['DAYS_EMPLOYED'].describe()

# DAYS_EMPLOYED 는 anomaly

In [ ]:
# EDA_7) anomaly data의 분포 확인

train['DAYS_EMPLOYED'].plot.hist(title = 'Days Employment Histogram')
test['DAYS_EMPLOYED'].plot.hist()
plt.xlabel('Days Employment')

test['DAYS_EMPLOYED'][test['DAYS_EMPLOYED'] > 200000]

In [ ]:
# EDA_8) anomaly data들의 TARGET 분포 확인

anomaly = train[train['DAYS_EMPLOYED'] == 365243]
non_anomaly = train[train['DAYS_EMPLOYED'] != 365243]

print('The non-anomalies default on %0.2f%% of loans' % (100 * non_anomaly['TARGET'].mean()))
print('The anomalies default on %0.2f%% of loans' % (100 * anomaly['TARGET'].mean()))
print('There are %d anomalous days of employment' % len(anomaly))

# anomaly의 TARGET이 5.4%로 더 낮음.
# 보통 anomaly를 다루기 위해 값을 그냥 채우지만, 다 같은 값으로 들어간다는 단점이 있어
# 임의의 값을 채우기로 함.

In [ ]:
# EDA_9) DAYS_EMPLOYED값이 anomaly인지 아닌지를 판단해주는 column 추가
train['DAYS_EMPLOYED_ANOM'] = train["DAYS_EMPLOYED"] == 365243
test['DAYS_EMPLOYED_ANOM'] = test["DAYS_EMPLOYED"] == 365243

# Anomaly 값들을 Nan값으로 replace
train['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace = True)
test["DAYS_EMPLOYED"].replace({365243: np.nan}, inplace = True)

train['DAYS_EMPLOYED'].plot.hist(title = 'Days Employment Histogram')
test['DAYS_EMPLOYED'].plot.hist()
plt.xlabel('Days Employment')

print('There are %d anomalies in the test data out of %d entries\n' % (test["DAYS_EMPLOYED_ANOM"].sum(), len(test)))

# test data의 경우 48744개의 data 중에 9274개의 data가 anomaly로 check

In [ ]:
# Correlation 관계 시각화

train_corr = train.corr()
plt.figure(figsize=(20, 12))
sns.heatmap(train_corr, square=True);
plt.show()

In [ ]:
# 변수가 너무 많아서 모든 변수간의 correlation을 파악하니 보기 좋게 만든 그래프이지만 해석이 불가능하다.
# 그래서 상위 10 개의 correlation을 파악하여 다시 시각화 진행

train_10_list = train_corr.nlargest(10, 'TARGET').index
top10_corrmat = np.corrcoef(train[train_10_list].values.T)
sns.set(font_scale=1.25)
plt.figure(figsize=(12, 9))
sns.heatmap(top10_corrmat, cbar=True, annot=True, square=True, fmt='.2f',
            annot_kws={'size': 10}, yticklabels=train_10_list.values, xticklabels=train_10_list.values)
plt.show()

In [ ]:
# EDA_10) TARGET과 변수들간의 상관관계 파악 후 정렬하여 positive/negative별로 정리(단순 나열)
correlations = train.corr()['TARGET'].sort_values()

# Display correlations
print('Positive한 상위 correlation :\n', correlations.tail(15))
print('\nNegative한 상위 correlation:\n', correlations.head(15))

# TARGET과 DAYS_BIRTH가 가장 양의 상관관계가 높게 나옴

In [ ]:
# 위 셀의 결과를 바탕으로 DAYS_BIRTH와 TARGET의 관계 확인
train['DAYS_BIRTH'] = abs(train['DAYS_BIRTH'])
train['DAYS_BIRTH'].corr(train['TARGET'])

In [ ]:
# 사전에 setting된 style이 있는 plot 사용
plt.style.use('fivethirtyeight')

# Age에 따른 고객의 분포 확인
plt.hist(train['DAYS_BIRTH'] / 365, edgecolor = 'k', bins = 25)
plt.title('Age of Client'); plt.xlabel('Age (years)'); plt.ylabel('Count')

# 3~40대에서 가장 많은 client가 있는 것을 확인할 수 있음.

In [ ]:
plt.figure(figsize = (10, 8))

# KDE plot of loans that were repaid on time
sns.kdeplot(train.loc[train['TARGET'] == 0, 'DAYS_BIRTH'] / 365, label = 'target == 0')

# KDE plot of loans which were not repaid on time
sns.kdeplot(train.loc[train['TARGET'] == 1, 'DAYS_BIRTH'] / 365, label = 'target == 1')

# Labeling of plot
plt.xlabel('Age (years)'); plt.ylabel('Density'); plt.title('Distribution of Ages')

# 상대적으로 어린 나이에 target value가 1인 것이 더 쏠려있는 것을 확인할 수 있음.
# 이는 추후에 있을 modeling에서 나이에 대해 가중치를 부여하는 것이 의미있을 수 있다는 것을 뜻함.

# 따라서, 이를 확인하기 위해 age 구간별로 통계를 분석해보고자 함.

## 나이대별로 데이터를 다루기 위한 기초 작업

In [ ]:
# train data에서 필요한 feature만 뽑아 정리
age_data = train[['TARGET', 'DAYS_BIRTH']]
age_data['YEARS_BIRTH'] = age_data['DAYS_BIRTH'] / 365

age_data['YEARS_BINNED'] = pd.cut(age_data['YEARS_BIRTH'], bins = np.linspace(20, 70, num = 11))
age_data.head(10)

In [ ]:
# 나잇대별로 그룹화를 시킨 후 평균값 계산
age_groups  = age_data.groupby('YEARS_BINNED').mean()
age_groups

In [ ]:
# 나잇대 별로 통계값 확인

plt.figure(figsize = (8, 8))

# Graph the age bins and the average of the target as a bar plot
plt.bar(age_groups.index.astype(str), 100 * age_groups['TARGET'])

# Plot labeling
plt.xticks(rotation = 75); plt.xlabel('Age Group'); plt.ylabel('No Repay (%)')
plt.title('Who do not repay(by Age group)')

# 나잇대가 어릴수록 do not repay할 가능성이 더 높다는 것이 확인됨.

In [ ]:
# 앞서 확인한 결과에서 Negative한 Correlation을 보여준 데이터들에 대해서 분석

ext_data = train[['TARGET', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH']]
ext_data_corrs = ext_data.corr()
ext_data_corrs

In [ ]:
# Correlation 시각화
plt.figure(figsize = (8, 6))

sns.heatmap(ext_data_corrs, cmap = plt.cm.RdYlBu_r, vmin = -0.25, annot = True, vmax = 0.6)
plt.title('Correlation Heatmap')

# 모든 Feature들이 TARGET과 음의 상관관계를 보여주고 있음
# 즉, EXT_SOURCE 값이 증가할수록 TARGET이 0이 될 가능성이 높다는 말.

In [ ]:
plt.figure(figsize = (10, 12))

for i, source in enumerate(['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']):    
    plt.subplot(3, 1, i + 1)
    sns.kdeplot(train.loc[train['TARGET'] == 0, source], label = 'target == 0')
    sns.kdeplot(train.loc[train['TARGET'] == 1, source], label = 'target == 1')
    
    plt.title('Distribution of %s (by Target)' % source)
    plt.xlabel('%s' % source); plt.ylabel('Density');
    
plt.tight_layout(h_pad = 2.5)

In [ ]:
# plot을 위해 data copy
plot_data=ext_data.drop(columns=['DAYS_BIRTH']).copy()

# 고객 나이 컬럼 추가
plot_data['YEARS_BIRTH']=age_data['YEARS_BIRTH']

# 결측치 drop
plot_data=plot_data.dropna().loc[:100000,:]

# 두 컬럼 간의 상관관계를 계산하는 함수 작성
def corr_func(x,y,**kwargs):
    r=np.corrcoef(x,y)[0][1]
    ax=plt.gca()
    ax.annotate("r={:.2f}".format(r),
               xy=(.2, .8),
               xycoords=ax.transAxes,
               size=20)

# vars = 변수명 리스트
grid=sns.PairGrid(data=plot_data, size=3, diag_sharey=False, hue='TARGET',
                 vars=plot_data.columns.drop(['TARGET']).tolist())

# 삼각형 위쪽 영역은 산점도
grid.map_upper(plt.scatter,alpha=0.2)

# 대각선은 히스토그램
grid.map_diag(sns.kdeplot)

# 삼각형 하단은 density plot
grid.map_lower(sns.kdeplot, cmap=plt.cm.OrRd_r);

plt.suptitle('Ext Source and Age Features Pairs Plot',size=32, y=1.05);

# YEAR_BIRTH와 EXT_SOURCE_1의 positive correaltion 파악 가능

## FE(add)

In [ ]:
# FE_4) Polynomial 방법 : data를 일차방정식 형태로 만들어줌
# 일차방정식 형태로 만들기 위해서는 여러가지 변수를 결합하여 새로운 형태의 변수로 만듦
# ex. A feature ^ 2*B feature^2

poly_features = train[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH', 'TARGET']]
poly_features_test = test[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH']]

# 결측치 처리를 위한 Library
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='median')

# target값 따로 저장
poly_target=poly_features['TARGET']

# target값 제외한 나머지 변수 저장 
poly_features=poly_features.drop(columns=['TARGET'])

# train/test 모두에 결측치 제거
poly_features=imputer.fit_transform(poly_features)
poly_features_test=imputer.transform(poly_features_test)

In [ ]:
poly_features

In [ ]:
poly_features_test

In [ ]:
# 앞서 말한 Polynomial을 위한 library import 및 적용
from sklearn.preprocessing import PolynomialFeatures

poly_transformer=PolynomialFeatures(degree=3)

In [ ]:
# train poly_features(polynomial data) data에 대해 fit
poly_transformer.fit(poly_features)

poly_features=poly_transformer.transform(poly_features)
poly_features_test=poly_transformer.transform(poly_features_test)
print('Polynomial Features shape: ', poly_features.shape)

# 35새의 feature가 생성된 것을 확인.

In [ ]:
# get_feature_names를 사용하면 polynomial을 통해 생성된 변수들을 확인할 수 있음

poly_transformer.get_feature_names(input_features=['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH'])[:15]

In [ ]:
# 새롭게 생성된 feature를 통해 dataframe 생성
poly_features = pd.DataFrame(poly_features, 
                             columns = poly_transformer.get_feature_names(['EXT_SOURCE_1', 'EXT_SOURCE_2', 
                                                                           'EXT_SOURCE_3', 'DAYS_BIRTH']))

# TARGET 추가
poly_features['TARGET']=poly_target

# TARGET과의 상관관계 확인
poly_corrs=poly_features.corr()['TARGET'].sort_values()

# 높은 양/음의 상관관계를 가진 feature 확인
print(poly_corrs.head(10))
print(poly_corrs.tail(5))

# 새로 만들어진 변수들 중 기존보다 상관관계가 높아진 것이 존재
# EXT_SOURCE_3 : -0.18 -> EXT_SOURCE_2 & EXT_SOURCE_3 -0.19

In [ ]:
# test data도 dataframe에 적용
poly_features_test = pd.DataFrame(poly_features_test, 
                                  columns = poly_transformer.get_feature_names(['EXT_SOURCE_1', 'EXT_SOURCE_2', 
                                                                                'EXT_SOURCE_3', 'DAYS_BIRTH']))

# 원본 train 데이터에 polynomial data merge로 새로운 데이터셋 생성
poly_features['SK_ID_CURR'] = train['SK_ID_CURR']
app_train_poly = train.merge(poly_features, on = 'SK_ID_CURR', how = 'left')

# 원본 test 데이터에 polynomial data merge로 새로운 데이터셋 생성
poly_features_test['SK_ID_CURR'] = test['SK_ID_CURR']
app_test_poly = test.merge(poly_features_test, on = 'SK_ID_CURR', how = 'left')

# dataframe align
app_train_poly, app_test_poly = app_train_poly.align(app_test_poly, join = 'inner', axis = 1)

# shape check
print('Training data with polynomial features shape: ', app_train_poly.shape)
print('Testing data with polynomial features shape:  ', app_test_poly.shape)

In [ ]:
# FE_5) Domain 지식을 통한 FE
# CREDIT_INCOME_PERCENT: income에 따른 credit amount의 비율
# ANNUITY_INCOME_PERCENT: income에 따른 loan annuity의 비율
# CREDIT_TERM: 월 납부 길이
# DAYS_EMPLOYED_PERCENT: age에 따른 dayes employed의 비율

In [ ]:
app_train_domain=train.copy()
app_test_domain=test.copy()

# train데이터에 새로운 변수 추가
app_train_domain['CREDIT_INCOME_PERCENT'] = app_train_domain['AMT_CREDIT'] / app_train_domain['AMT_INCOME_TOTAL']
app_train_domain['ANNUITY_INCOME_PERCENT'] = app_train_domain['AMT_ANNUITY'] / app_train_domain['AMT_INCOME_TOTAL']
app_train_domain['CREDIT_TERM'] = app_train_domain['AMT_ANNUITY'] / app_train_domain['AMT_CREDIT']
app_train_domain['DAYS_EMPLOYED_PERCENT'] = app_train_domain['DAYS_EMPLOYED'] / app_train_domain['DAYS_BIRTH']

In [ ]:
# test데이터에 새로운 변수 추가
app_test_domain['CREDIT_INCOME_PERCENT'] = app_test_domain['AMT_CREDIT'] / app_test_domain['AMT_INCOME_TOTAL']
app_test_domain['ANNUITY_INCOME_PERCENT'] = app_test_domain['AMT_ANNUITY'] / app_test_domain['AMT_INCOME_TOTAL']
app_test_domain['CREDIT_TERM'] = app_test_domain['AMT_ANNUITY'] / app_test_domain['AMT_CREDIT']
app_test_domain['DAYS_EMPLOYED_PERCENT'] = app_test_domain['DAYS_EMPLOYED'] / app_test_domain['DAYS_BIRTH']

In [ ]:
# Domain을 통해 만들어진 새로운 변수에 대한 시각화

plt.figure(figsize=(12,20))
for i, feature in enumerate(['CREDIT_INCOME_PERCENT','ANNUITY_INCOME_PERCENT','CREDIT_TERM','DAYS_EMPLOYED_PERCENT']):
    plt.subplot(4,1,i+1)
    sns.kdeplot(app_train_domain.loc[app_train_domain['TARGET']==0,feature],label='target==0')
    sns.kdeplot(app_train_domain.loc[app_train_domain['TARGET']==1,feature],label='target==1')
    
    plt.title('Distribution of %s by Target Value' % feature)
    plt.xlabel('%s' % feature);
    plt.ylabel('Density');
    
plt.tight_layout(h_pad=2.5)

# TARGET에 따른 분포 차이가 크지 않음을 확인.

## Modeling

In [ ]:
train_ids = train['SK_ID_CURR']
test_ids = test['SK_ID_CURR']

labels = train['TARGET']

train = train.drop(columns = ['SK_ID_CURR', 'TARGET'])
test = test.drop(columns = ['SK_ID_CURR'])

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer

# TARGET은 결측치 및 Scaling 대상이 아니라서 drop 
if 'TARGET' in train:
    train=train.drop(columns=['TARGET'])
else:
    train=train.copy()

# 변수이름
features=list(train.columns)

# testing 데이터 복사
test=test.copy()

# 결측치를 median값으로 처리
imputer = SimpleImputer(strategy='median')

# Normalization
scaler=MinMaxScaler(feature_range=(0,1))

# training 데이터에 fit
imputer.fit(train)

# training데이터와 testing데이터에 둘다 transform
## imputer 처리 하고나면 DataFrame에서 array형태로 바뀜
train=imputer.transform(train)
test=imputer.transform(test)

# Scaling
scaler.fit(train)
train=scaler.transform(train)
test=scaler.transform(test)

print('Training data shape: ', train.shape)
print('Testing data shape: ', test.shape)

In [ ]:
from sklearn.model_selection import train_test_split
train_x, valid_x, train_y, valid_y = train_test_split(train, labels, test_size=0.3, random_state=2020)

In [ ]:
print(train_x.shape, valid_x.shape)

In [ ]:
from lightgbm import LGBMClassifier
clf = LGBMClassifier(
            n_jobs=-1,
            n_estimators=2000,
            learning_rate=0.02,
            max_depth = 11,
            num_leaves=58,
            silent=-1,
            subsample=0.708,
            reg_alpha=3.564,
            reg_lambda=4.930,
            verbose=-1
            )

clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], eval_metric= 'auc',
        verbose= 100, early_stopping_rounds= 100)

In [ ]:
preds = clf.predict_proba(test, axis=1)[:, 1 ]

In [ ]:
from lightgbm import plot_importance

plot_importance(clf, figsize=(16, 32))

In [ ]:
submission = pd.DataFrame({'SK_ID_CURR': test_ids, 'TARGET': preds})

In [ ]:
submission.head()

In [ ]:
submission.to_csv('BigData_project_EDA_FE(18_cs_Naeun).csv', index=False)